In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

def load_csv(path):
    return pd.read_csv(path)

def clean_text_columns(df, cols):
    for col in cols:
        df[col] = df[col].astype(str).str.strip().str.lower()
    return df

def fill_missing_values(df):
    if 'duration_ms' in df.columns:
        df['duration_ms'] = df['duration_ms'].fillna(df['duration_ms'].median())
    if 'audio_sample_rate' in df.columns:
        df['audio_sample_rate'] = df['audio_sample_rate'].fillna(df['audio_sample_rate'].mode()[0])
    return df

def convert_to_datetime(df, column):
    df[column] = pd.to_datetime(df[column], errors='coerce')
    return df

def drop_columns(df, cols_to_drop):
    return df.drop(columns=cols_to_drop)

def scale_minmax(df, columns_to_scale):
    scaler = MinMaxScaler()
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
    return df

def merge_on_mbid(df1, df2):
    df = df1.merge(df2, on='mbid', how='inner')
    df = df.drop(columns=['title_y', 'artist_y', 'genre_y', 'year_y', 'duration_ms_y'])
    df = df.rename(columns={
        'title_x': 'title', 'artist_x': 'artist', 'genre_x': 'genre',
        'year_x': 'year', 'duration_ms_x': 'duration_ms'
    })
    return df

def encode_binary_column(df, column, positive_val='danceable', negative_val='not_danceable'):
    df[column + '_encoded'] = df[column].map({positive_val: 1, negative_val: 0})
    return df.drop(columns=[column])

def encode_binary_multiple(df, binary_cols):
    for col in binary_cols:
        unique_vals = df[col].dropna().unique()
        if len(unique_vals) == 2:
            df[col] = df[col].map({unique_vals[0]: 0, unique_vals[1]: 1})
    return df

def one_hot_encode(df, cols):
    return pd.get_dummies(df, columns=cols, drop_first=True)

def frequency_encode(df, cols):
    for col in cols:
        freq_map = df[col].value_counts().to_dict()
        df[col] = df[col].map(freq_map)
    return df

def final_cleanup(df):
    if 'release_date' in df.columns:
        df['release_year'] = df['release_date'].dt.year
        df['release_year'].fillna(df['release_year'].median(), inplace=True)
        df.drop(columns='release_date', inplace=True)

    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    for col in num_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].median())

    cat_cols = df.select_dtypes(include='object').columns
    for col in cat_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].mode()[0])

    return df

def scale_standard(df):
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    scaler = StandardScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    return df


In [3]:
# Cargar datasets
pop_df = load_csv("popularity.csv")
feat_df = load_csv("features_full_final.csv")

# Limpiar texto
pop_df = clean_text_columns(pop_df, ['title', 'artist', 'genre'])
feat_df = clean_text_columns(feat_df, ['title', 'artist', 'genre'])

# Rellenar nulos
pop_df = fill_missing_values(pop_df)
feat_df = fill_missing_values(feat_df)

# Convertir fechas y eliminar columnas irrelevantes
pop_df = convert_to_datetime(pop_df, 'release_date')
pop_df = drop_columns(pop_df, ['artist_followers', 'extraction_date', 'artist_popularity', 'artist_id', 'spotify_id'])

# Escalar variables numéricas con MinMaxScaler
pop_df = scale_minmax(pop_df, ['duration_ms', 'year', 'popularity'])

# Unir datasets por 'mbid'
df = merge_on_mbid(pop_df, feat_df)

# Codificación binaria personalizada
if 'high_danceability_value' in df.columns:
    df = encode_binary_column(df, 'high_danceability_value', 'danceable', 'not_danceable')

# Codificación binaria
binary_cols = [
    'high_gender_value', 'high_mood_acoustic_value', 'high_mood_aggressive_value',
    'high_mood_electronic_value', 'high_mood_happy_value', 'high_mood_party_value',
    'high_mood_relaxed_value', 'high_mood_sad_value', 'high_timbre_value',
    'high_tonal_atonal_value', 'high_voice_instrumental_value', 'audio_downmix',
    'low_key_scale', 'low_chords_scale'
]
df = encode_binary_multiple(df, binary_cols)

# One-hot encoding a columnas seleccionadas
df = one_hot_encode(df, ['genre', 'high_genre_electronic_value', 'high_moods_mirex_value'])

# Codificación por frecuencia para columnas con cardinalidad media
medium_card_cols = [
    'high_genre_dortmund_value', 'high_genre_rosamerica_value',
    'high_genre_tzanetakis_value', 'high_ismir04_rhythm_value',
    'audio_codec', 'low_key_key', 'low_chords_key'
]
df = frequency_encode(df, medium_card_cols)

# Limpieza final (release year, nulos restantes)
df = final_cleanup(df)

# Escalar todas las columnas numéricas con StandardScaler
df = scale_standard(df)

# Vista previa final
df.head()


/var/folders/bx/0fhb_jz13rd9tr24kwrp9cf00000gn/T/ipykernel_58642/1078287089.py:6: DtypeWarning: Columns (9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path)
/var/folders/bx/0fhb_jz13rd9tr24kwrp9cf00000gn/T/ipykernel_58642/1078287089.py:64: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['release_year'].fillna(df['release_year'].median(), inplace=True)


,mbid,title,artist,duration_ms,year,popularity,high_danceability_probability,high_gender_value,high_gender_probability,high_genre_dortmund_value,...,genre_rock,high_genre_electronic_value_dnb,high_genre_electronic_value_house,high_genre_electronic_value_techno,high_genre_electronic_value_trance,high_moods_mirex_value_Cluster2,high_moods_mirex_value_Cluster3,high_moods_mirex_value_Cluster4,high_moods_mirex_value_Cluster5,release_year
0,50971422-fed2-47ad-9a6f-eacf64ebb965,shades of green (utah saints 7″ remix),the mission,-0.138804,-1.205936,-1.116926,0.750090,-1.259873,0.388071,0.361673,...,True,False,False,False,True,False,False,False,True,-3.600311
1,76c66068-5d89-45c2-92be-22f7562dfa3c,mr. pleasant,the mission,-0.594273,-1.205936,-0.479845,-2.499189,-1.259873,1.384531,0.361673,...,True,False,False,False,False,False,False,True,False,-6.803515
2,e9da3272-429d-41a2-b84d-b0c09b99a286,118,mono inc.,0.089969,-1.205936,-0.822889,-0.253298,0.793731,0.759693,0.361673,...,True,False,False,False,True,False,False,False,True,-0.797507
3,86f3c7ed-ad1b-4fc8-9679-a89fb2a50435,someday somewhere somehow,to/die/for,0.949307,-1.205936,-0.969908,-0.531315,-1.259873,-1.598128,0.361673,...,True,False,False,False,True,False,False,False,True,-2.399109
4,9c45c16f-addc-4a53-ad2b-f5481556bfcb,folie á deux,to/die/for,1.168642,-1.205936,-0.626864,0.070998,-1.259873,0.258050,0.361673,...,True,False,False,False,False,False,True,False,False,-2.399109


In [4]:
df.to_csv("/Users/andreagrain/code/PolloDK/Hitanalyzer/datos_limpios_02.csv", index=False)